In [9]:
import requests
from bs4 import BeautifulSoup
import json
import re
import sys
import time, random
import pandas as pd

header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

def get_news(news_url):
    news_detail = []
    print(news_url)
    req = requests.get(news_url, headers=header)
    soup = BeautifulSoup(req.content, 'html.parser')

    # 제목
    title = soup.select('h3#articleTitle')[0].text
    news_detail.append(title)

    # 날짜 
    pdate = soup.select('.t11')[0].get_text()[:11]
    news_detail.append(pdate)

    # 본문
    _text = soup.select('#articleBodyContents')[
        0].get_text().replace('\n', " ")
    text = _text.replace(
        "// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(text.strip())

    # 언론사
    pcompany = soup.select('div.article_footer')[
        0].a.get_text().split()[0].strip()
    news_detail.append(pcompany)

    return news_detail

columns = ['날짜', '신문사', '제목', '내용']
df = pd.DataFrame(columns=columns)

query = '코로나'   # url 인코딩 에러는 encoding parse.quote(query)
s_date = "2022.04.08"
e_date = "2022.04.14"
s_from = s_date.replace(".", "")
e_to = e_date.replace(".", "")
page = 1

while True:
    time.sleep(random.sample(range(3), 1)[0])
    print(page)

    url = "https://search.naver.com/search.naver?where=news&query=" + query + \
        "&sort=1&field=1&ds=" + s_date + "&de=" + e_date + \
        "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + \
        "%2Ca%3A&start=" + str(page)

    req = requests.get(url, headers=header)
    print(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')

    naver_news = soup.find_all("a", {"class": "info"})
    if naver_news == []:
        break

    for a_tag in naver_news:
        try:
            news_url = a_tag.attrs["href"]
            if news_url.startswith("https://news.naver.com"):
                print(news_url)
                news_detail = get_news(news_url)
                print(news_detail)
                #df = df.append(pd.DataFrame(
                #    [[news_detail[1], news_detail[3], news_detail[0], news_detail[2]]], columns=columns))
        except Exception as e:
            print(e)
            continue
    break
    
    page += 10

1
https://search.naver.com/search.naver?where=news&query=코로나&sort=1&field=1&ds=2022.04.08&de=2022.04.14&nso=so%3Ar%2Cp%3Afrom20220408to20220414%2Ca%3A&start=1
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=001&aid=0013113075
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=001&aid=0013113075
["코로나19 이후 2년 만에 해제된 '특별여행주의보'", '2022.04.14.', '(영종도=연합뉴스) 김도훈 기자 = 정부가 코로나19 확산 이후 2년여 만에 특별여행주의보를 해제한 14일 오전 인천국제공항 제1여객터미널 출국장에서 탑승객들이 출국 수속을 밟고 있다.         이에 따라 싱가포르, 캐나다 등 22개 나라와 괌과 사이판 등 2개 지역은 여행경보 1단계로 하향 조정되며, 미국 본토, 영국, 독일 등 129개 나라는 여행경보 2단계로 조정된다.    한편 중국과 러시아 등 26개국은 특별여행주의보가 유지되며, 기존 여행경보 3단계(출국권고) 및 4단계(여행금지) 발령 국가와 지역은 현 단계가 유지된다. 2022.4.14    superdoo82@yna.co.kr', '연합뉴스']
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=001&aid=0013113079
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=001&aid=0013113079
["코로나19 이후 2년만 해제된 '특별여행주의보'", '2022.04.14.', '(영종도=연합뉴스) 김도훈 기자 = 정부가 코로나19 확산 이후 2년여 만에 특별

In [11]:
news_detail = get_news("https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=001&aid=0013113075")
print(news_detail)

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=001&aid=0013113075
["코로나19 이후 2년 만에 해제된 '특별여행주의보'", '2022.04.14.', '(영종도=연합뉴스) 김도훈 기자 = 정부가 코로나19 확산 이후 2년여 만에 특별여행주의보를 해제한 14일 오전 인천국제공항 제1여객터미널 출국장에서 탑승객들이 출국 수속을 밟고 있다.         이에 따라 싱가포르, 캐나다 등 22개 나라와 괌과 사이판 등 2개 지역은 여행경보 1단계로 하향 조정되며, 미국 본토, 영국, 독일 등 129개 나라는 여행경보 2단계로 조정된다.    한편 중국과 러시아 등 26개국은 특별여행주의보가 유지되며, 기존 여행경보 3단계(출국권고) 및 4단계(여행금지) 발령 국가와 지역은 현 단계가 유지된다. 2022.4.14    superdoo82@yna.co.kr', '연합뉴스']
